In [61]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from datetime import date
from bs4 import BeautifulSoup
import sys
import time
import pandas as pd
import datetime

now = datetime.datetime.now().strftime("%Y%m%d")

In [62]:
def listToString(s): 
    str1 = "" 
    for ele in s: 
        str1 += ele  
    return str1 

In [63]:
#new chrome driver
def OpenChromeDriver():
    #開啟瀏覽器新分頁，下載google chrome driver
    option = webdriver.ChromeOptions()

    #fix chrome 103 problem
    option.binary_location = "/Applications/Google Chrome Beta.app/Contents/MacOS/Google Chrome Beta"

    prefs={"profile.default_content_setting_values.notifications" : 2}
    option.add_experimental_option("prefs",prefs)
    gchrome=webdriver.Chrome(options=option)
    return gchrome

def FinishedCrawler(driver):
    driver.close()

In [64]:
#抓取幾頁
def getPageNum(url,driver):
    driver.get(url)
            
    #Get Comment       
    soup = BeautifulSoup(driver.page_source,"html.parser")
    pages = soup.find("div", class_="article_multi_page clear")
    try:
        page_num = len(pages.find_all("a"))
    except:
        return 1
    else:
        return page_num 

In [65]:
#抓取貼文連結
def getPostLink(url,driver):
    driver.get(url)
    
    try:
        for i in range(0,5):
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
            i += 1
            time.sleep(2)
    except OSError:
        time.sleep(15)
        
        for i in range(0,5):
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
            i += 1
            time.sleep(2)   
    post_link_list = []
    soup = BeautifulSoup(driver.page_source,"html.parser")
    for l in soup.find_all("div", class_="content_col"):
        temp_soup = BeautifulSoup(str(l),"html.parser")
        temp_target = temp_soup.find("a")
        temp = []
        temp.append(temp_target.get_text().strip().split('\t'))
        temp.append("https://www.dcfever.com"+temp_target["href"])
        post_link_list.append(temp)
    link_df = pd.DataFrame(data=post_link_list, columns=['Post Title', 'Link'])
    return link_df

In [66]:
#取得貼文日期

def getPostDate(page_soup):
    date_tag_string = page_soup.find("div", class_="article_date").get_text()
    date_tag_string = date_tag_string[date_tag_string.find("發表於")+3:date_tag_string.find("日")]
    year = date_tag_string[:date_tag_string.find("年")].strip()
    month = date_tag_string[date_tag_string.find("年")+1:date_tag_string.find("月")].strip()
    day = date_tag_string[date_tag_string.find("月")+1:].strip()
    post_date = datetime.date(int(year),int(month),int(day))
    return post_date.strftime("%Y/%m/%d")


In [67]:
#抓取貼文留言
def getPostContent(first_url,driver):
    
    urls=[]
    content_all=[]
    post_date = ""
    
    page_num = getPageNum(first_url,driver)
    if (page_num>1):
        for i in range(1,page_num+1):
            urls.append(first_url+"&page="+str(i))
        for url in urls:
            driver.get(url)
        #Get Comment       
            soup = BeautifulSoup(driver.page_source,"html.parser")
            if (url.find("&page=1") != -1):
                post_date = getPostDate(soup)
        
        content_all+=soup.find("div",class_="article_content").get_text(strip=True)
    
    else:
        driver.get(first_url)
        soup = BeautifulSoup(driver.page_source,"html.parser")
        post_date = getPostDate(soup)
        content_all+=soup.find("div",class_="article_content").get_text(strip=True)

    FinishedCrawler(driver)
    return listToString(content_all), post_date

In [68]:
# 兩部份合併
search_part = getPostLink('https://www.dcfever.com/phones/articles.php?id=2922', OpenChromeDriver())
news_part = getPostLink('https://www.dcfever.com/news/search.php?keyword=iphone13&form_action=search_action&search_btn=%E6%90%9C%E5%B0%8B', OpenChromeDriver())
# 合併+刪重複值+reset_index
link_df = pd.concat([search_part,news_part],axis=0)
link_df = link_df[~link_df['Post Title'].apply(tuple).duplicated()].reset_index(drop=True)
link_df

,Post Title,Link
0,[【iOS 15.4 新功能】iPhone 13 Pro 戴口罩使用 Face ID 實試],https://www.dcfever.com/news/readnews.php?id=3...
1,[iPhone iOS 15.1 更新：新增 ProRes 影片及 FaceTime 同播同享],https://www.dcfever.com/news/readnews.php?id=3...
2,[iPhone Leather MagSafe Wallet 試玩：點只支援「尋找」咁簡單],https://www.dcfever.com/news/readnews.php?id=3...
3,[iPhone 13 炒價直擊：今年都是開紅盤],https://www.dcfever.com/news/readnews.php?id=3...
4,[iPhone 13 系列首位用家：鍾意微距拍攝方便影食物],https://www.dcfever.com/news/readnews.php?id=3...
5,[iPhone 13 系列開箱 (連影片)：看看跟上代有什麼不同],https://www.dcfever.com/news/readnews.php?id=3...
6,[iPhone 13 系列兩個因素很大機會炒燶],https://www.dcfever.com/news/readnews.php?id=3...
7,[iPhone 13 五台出機優惠全集],https://www.dcfever.com/news/readnews.php?id=3...
8,[【攝力分析】iPhone 13、iPhone 13 Pro 系列看似平淡但進步很大],https://www.dcfever.com/news/readnews.php?id=3...
9,[iPhone 13 系列炒價分析：或是消費券套現最佳工具],https://www.dcfever.com/news/readnews.php?id=3...


In [69]:
# 先把爬下來的主題跟網址存起來
file_name = "crawler_data/DCfever_iphone13_Link"+now+".csv"
link_df.to_csv(file_name, index=False, encoding="utf-8")

In [70]:
contents = []
dates = []
for link in link_df.loc[:,"Link"]:
    temp_content, temp_date = getPostContent(link,OpenChromeDriver())
    contents.append(temp_content)
    dates.append(temp_date)

content_df = pd.DataFrame({'Content': contents,'Date': dates})

In [71]:
all_df = link_df.join(content_df)
all_df

,Post Title,Link,Content,Date
0,[【iOS 15.4 新功能】iPhone 13 Pro 戴口罩使用 Face ID 實試],https://www.dcfever.com/news/readnews.php?id=3...,文：TonyApple 在今日凌晨釋出 iOS 15.4 版本，加入了戴口罩使用 Face ...,2022/03/15
1,[iPhone iOS 15.1 更新：新增 ProRes 影片及 FaceTime 同播同享],https://www.dcfever.com/news/readnews.php?id=3...,文：TonyApple 剛剛釋出 iOS 15.1 更新，主要迎來三大功能更新，包括 Fac...,2021/10/26
2,[iPhone Leather MagSafe Wallet 試玩：點只支援「尋找」咁簡單],https://www.dcfever.com/news/readnews.php?id=3...,文：TonyApple 為iPhone 13系列推出不少全新配件，當中同樣支援iPhone ...,2021/10/06
3,[iPhone 13 炒價直擊：今年都是開紅盤],https://www.dcfever.com/news/readnews.php?id=3...,文：Tony每年 iPhone 的新機推出，除新功能外，大家都想八卦一下有沒有炒價，想知新功...,2021/09/24
4,[iPhone 13 系列首位用家：鍾意微距拍攝方便影食物],https://www.dcfever.com/news/readnews.php?id=3...,文：TonyiPhone 13系列今日正式推出，SmarTone 一如以往都會提早開門營業，...,2021/09/24
5,[iPhone 13 系列開箱 (連影片)：看看跟上代有什麼不同],https://www.dcfever.com/news/readnews.php?id=3...,文：TonyiPhone 13系列今日正式推出，為大家詳細測試之前，先送上開箱文，讓大家看看...,2021/09/24
6,[iPhone 13 系列兩個因素很大機會炒燶],https://www.dcfever.com/news/readnews.php?id=3...,文：TonyiPhone 13 系列的炒價，除了果農關心之外，也是大家茶餘飯後的話題。想知i...,2021/09/20
7,[iPhone 13 五台出機優惠全集],https://www.dcfever.com/news/readnews.php?id=3...,文：TonyiPhone 13系列今日起正式預售，各電訊商亦推出上台或購機優惠，大家可以看看...,2021/09/17
8,[【攝力分析】iPhone 13、iPhone 13 Pro 系列看似平淡但進步很大],https://www.dcfever.com/news/readnews.php?id=3...,文：TonyiPhone 13系列經已發表，iPhone 13與iPhone 13 mini...,2021/09/17
9,[iPhone 13 系列炒價分析：或是消費券套現最佳工具],https://www.dcfever.com/news/readnews.php?id=3...,文：TonyiPhone 13 系統昨日公佈，大家除了較關心新機有什麼地方有所升級 (可看懶...,2021/09/16


In [72]:
file_name = "crawler_data/DCfever_iphone13_Data"+now+".csv"
all_df.to_csv(file_name, index=False, encoding="utf-8")